Diretoria

In [ ]:
select	d.IdDiretoria, d.Sigla, d.NomeDiretoria,
		GD.IdDiretoria, gd.Sigla, gd.NomeDiretoria, 
		PF.NomePessoaFisica,
		PF2.NomePessoaFisica
from GenDiretoria D
LEFT JOIN Gen_Diretoria GD ON d.IdDiretoria = gd.IdDiretoria
LEFT JOIN Gen_PessoaFisica PF ON IdOperadorDiretor = IdPessoaFisica
LEFT JOIN Gen_PessoaFisica PF2 ON IdOperadorDiretorSubstituto = PF2.IdPessoaFisica

WHERe ( d.Sigla <> gd.Sigla ) OR ( d.IdDiretoria <> GD.IdDiretoria ) OR ( d.NomeDiretoria <> GD.NomeDiretoria )
ORDER BY d.Sigla

In [ ]:
SELECT count(*) FROM GenDiretoria --d WHERE NOT EXISTS( SELECT * FROM Gen_Diretoria gd  WHERE d.IdDiretoria = gd.IdDiretoria )
SELECT * FROM Gen_Diretoria gd WHERE NOT EXISTS( SELECT * FROM GenDiretoria d  WHERE d.IdDiretoria = gd.IdDiretoria )

SELECT COUNT(*) FROM Setor
SELECT COUNT(*) FROM Gen_Setor

In [ ]:
select	s.IdSetor, s.Codigo CodigoAR, s.nome NomeAR, s.DataInclusao, s.IdDiretoria IdDiretoriaAR,
		gs.IdSetor, CodigoSetor CodigoSetorBDC, NomeSetor, GS.IdDiretoria
from Setor S
LEFT JOIN Gen_Setor GS ON s.codigo = gs.CodigoSetor
--LEFT JOIN Gen_Setor GS ON s.IdSetor = gs.idSetor
WHERe 
--	( s.nome <> NomeSetor ) 
--	( s.IdDiretoria <> GS.IdDiretoria)

-- ( s.IdSetor <> GS.IdSetor) -- MANUALMENTE, APPOS DESLIGAR IDENTITY 

	( CodigoSetor IS NULL )  -- NOVOS

ORDER BY Codigo


In [ ]:
begin tran

UPDATE Gen_Setor 
SET Gen_Setor.NomeSetor = Setor.Nome
    FROM Setor
    WHERE Gen_Setor.IdSetor = Setor.IdSetor AND Setor.nome <> Gen_Setor.NomeSetor
 
-- rollback
-- commit


begin tran

UPDATE Gen_Setor 
SET Gen_Setor.IdDiretoria = Setor.IdDiretoria
    FROM Setor
    WHERE Gen_Setor.IdSetor = Setor.IdSetor AND Setor.idDiretoria <> Gen_Setor.IdDiretoria


-- rollback
-- commit

In [ ]:
BEGIN TRAN

UPDATE Gen_Setor
	SET IdSetor = 609
where IdSetor = 605

UPDATE Gen_Setor
	SET IdSetor = 610
where IdSetor = 606

UPDATE Gen_Setor
	SET IdSetor = 611
where IdSetor = 607
-- commit
-- rollback

In [ ]:
-- DESLIGAR IDENTITY Gen_Setor


/* To prevent any potential data loss issues, you should review this script in detail before running it outside the context of the database designer.*/
BEGIN TRANSACTION
SET QUOTED_IDENTIFIER ON
SET ARITHABORT ON
SET NUMERIC_ROUNDABORT OFF
SET CONCAT_NULL_YIELDS_NULL ON
SET ANSI_NULLS ON
SET ANSI_PADDING ON
SET ANSI_WARNINGS ON
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT FK_Gen_Setor_Gen_Setor_IdSessao
GO
ALTER TABLE dbo.Seg_Sessao SET (LOCK_ESCALATION = TABLE)
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT FK_Gen_Setor_Gen_Diretoria_IdDiretoria
GO
ALTER TABLE dbo.Gen_Diretoria SET (LOCK_ESCALATION = TABLE)
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_SetorFazAnalise
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_SetorTramita
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_SetorTramitaSetorExterno
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_TrabalhaProcessoEletronico
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_FazDistribuicao
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_SetorBloqueado
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_DataInclusao
GO
CREATE TABLE dbo.Tmp_Gen_Setor
	(
	IdSetor int NOT NULL,
	IdDiretoria int NOT NULL,
	CodigoSetor varchar(10) NOT NULL,
	NomeSetor varchar(100) NOT NULL,
	AreaSetor char(1) NULL,
	SetorFazAnalise bit NOT NULL,
	SetorTramita bit NOT NULL,
	SetorTramitaSetorExterno bit NOT NULL,
	TrabalhaProcessoEletronico bit NOT NULL,
	FazDistribuicao bit NOT NULL,
	IdOperadorResponsavelPonto int NULL,
	IdAvaliadorDiretor int NULL,
	SetorBatePonto bit NOT NULL,
	SetorAtivo bit NOT NULL,
	DataInclusao smalldatetime NOT NULL,
	IdSessao int NOT NULL,
	IdSessaoOperacao int NULL
	)  ON [PRIMARY]
GO
ALTER TABLE dbo.Tmp_Gen_Setor SET (LOCK_ESCALATION = TABLE)
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_SetorFazAnalise DEFAULT ((0)) FOR SetorFazAnalise
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_SetorTramita DEFAULT ((0)) FOR SetorTramita
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_SetorTramitaSetorExterno DEFAULT ((0)) FOR SetorTramitaSetorExterno
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_TrabalhaProcessoEletronico DEFAULT ((0)) FOR TrabalhaProcessoEletronico
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_FazDistribuicao DEFAULT ((0)) FOR FazDistribuicao
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_SetorBloqueado DEFAULT ((1)) FOR SetorAtivo
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_DataInclusao DEFAULT (getdate()) FOR DataInclusao
GO
IF EXISTS(SELECT * FROM dbo.Gen_Setor)
	 EXEC('INSERT INTO dbo.Tmp_Gen_Setor (IdSetor, IdDiretoria, CodigoSetor, NomeSetor, AreaSetor, SetorFazAnalise, SetorTramita, SetorTramitaSetorExterno, TrabalhaProcessoEletronico, FazDistribuicao, IdOperadorResponsavelPonto, IdAvaliadorDiretor, SetorBatePonto, SetorAtivo, DataInclusao, IdSessao, IdSessaoOperacao)
		SELECT IdSetor, IdDiretoria, CodigoSetor, NomeSetor, AreaSetor, SetorFazAnalise, SetorTramita, SetorTramitaSetorExterno, TrabalhaProcessoEletronico, FazDistribuicao, IdOperadorResponsavelPonto, IdAvaliadorDiretor, SetorBatePonto, SetorAtivo, DataInclusao, IdSessao, IdSessaoOperacao FROM dbo.Gen_Setor WITH (HOLDLOCK TABLOCKX)')
GO
ALTER TABLE dbo.Seg_OperadorInterno
	DROP CONSTRAINT FK_Seg_OperadorInterno_Gen_Setor_IdSetor_IdSetorLotacao
GO
ALTER TABLE dbo.Seg_AutorizacaoSetorPerfil
	DROP CONSTRAINT FK_Seg_AutorizacaoSetorPerfil_Gen_Setor_IdSetor
GO
ALTER TABLE dbo.Seg_OperadorInternoSetor
	DROP CONSTRAINT FK_Seg_OperadorInternoSetor_Gen_Setor_IdSetor
GO
ALTER TABLE dbo.Gen_UnidadeJurisdicionada
	DROP CONSTRAINT FK_Gen_UnidadeJurisdicionada_Gen_Setor_IdSetor
GO
DROP TABLE dbo.Gen_Setor
GO
EXECUTE sp_rename N'dbo.Tmp_Gen_Setor', N'Gen_Setor', 'OBJECT' 
GO
ALTER TABLE dbo.Gen_Setor ADD CONSTRAINT
	PK_Gen_Setor PRIMARY KEY CLUSTERED 
	(
	IdSetor
	) WITH( STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]

GO
CREATE UNIQUE NONCLUSTERED INDEX AK_Gen_Setor ON dbo.Gen_Setor
	(
	CodigoSetor
	) WITH( STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
GO
CREATE NONCLUSTERED INDEX IX_Gen_Setor_NomeSetor ON dbo.Gen_Setor
	(
	NomeSetor
	) WITH( STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
GO
ALTER TABLE dbo.Gen_Setor ADD CONSTRAINT
	FK_Gen_Setor_Gen_Diretoria_IdDiretoria FOREIGN KEY
	(
	IdDiretoria
	) REFERENCES dbo.Gen_Diretoria
	(
	IdDiretoria
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
ALTER TABLE dbo.Gen_Setor ADD CONSTRAINT
	FK_Gen_Setor_Gen_Setor_IdSessao FOREIGN KEY
	(
	IdSessao
	) REFERENCES dbo.Seg_Sessao
	(
	IdSessao
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Gen_UnidadeJurisdicionada ADD CONSTRAINT
	FK_Gen_UnidadeJurisdicionada_Gen_Setor_IdSetor FOREIGN KEY
	(
	IdSetorResponsavel
	) REFERENCES dbo.Gen_Setor
	(
	IdSetor
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
ALTER TABLE dbo.Gen_UnidadeJurisdicionada SET (LOCK_ESCALATION = TABLE)
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Seg_OperadorInternoSetor ADD CONSTRAINT
	FK_Seg_OperadorInternoSetor_Gen_Setor_IdSetor FOREIGN KEY
	(
	IdSetor
	) REFERENCES dbo.Gen_Setor
	(
	IdSetor
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
ALTER TABLE dbo.Seg_OperadorInternoSetor SET (LOCK_ESCALATION = TABLE)
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Seg_AutorizacaoSetorPerfil ADD CONSTRAINT
	FK_Seg_AutorizacaoSetorPerfil_Gen_Setor_IdSetor FOREIGN KEY
	(
	IdSetor
	) REFERENCES dbo.Gen_Setor
	(
	IdSetor
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
ALTER TABLE dbo.Seg_AutorizacaoSetorPerfil SET (LOCK_ESCALATION = TABLE)
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Seg_OperadorInterno ADD CONSTRAINT
	FK_Seg_OperadorInterno_Gen_Setor_IdSetor_IdSetorLotacao FOREIGN KEY
	(
	IdSetorLotacao
	) REFERENCES dbo.Gen_Setor
	(
	IdSetor
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
ALTER TABLE dbo.Seg_OperadorInterno SET (LOCK_ESCALATION = TABLE)
GO
COMMIT



In [ ]:
-- RELIGAR IDENTITY Gen_Setor



/* To prevent any potential data loss issues, you should review this script in detail before running it outside the context of the database designer.*/
BEGIN TRANSACTION
SET QUOTED_IDENTIFIER ON
SET ARITHABORT ON
SET NUMERIC_ROUNDABORT OFF
SET CONCAT_NULL_YIELDS_NULL ON
SET ANSI_NULLS ON
SET ANSI_PADDING ON
SET ANSI_WARNINGS ON
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT FK_Gen_Setor_Gen_Setor_IdSessao
GO
ALTER TABLE dbo.Seg_Sessao SET (LOCK_ESCALATION = TABLE)
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT FK_Gen_Setor_Gen_Diretoria_IdDiretoria
GO
ALTER TABLE dbo.Gen_Diretoria SET (LOCK_ESCALATION = TABLE)
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_SetorFazAnalise
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_SetorTramita
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_SetorTramitaSetorExterno
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_TrabalhaProcessoEletronico
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_FazDistribuicao
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_SetorBloqueado
GO
ALTER TABLE dbo.Gen_Setor
	DROP CONSTRAINT DF_Gen_Setor_DataInclusao
GO
CREATE TABLE dbo.Tmp_Gen_Setor
	(
	IdSetor int NOT NULL IDENTITY (1, 1),
	IdDiretoria int NOT NULL,
	CodigoSetor varchar(10) NOT NULL,
	NomeSetor varchar(100) NOT NULL,
	AreaSetor char(1) NULL,
	SetorFazAnalise bit NOT NULL,
	SetorTramita bit NOT NULL,
	SetorTramitaSetorExterno bit NOT NULL,
	TrabalhaProcessoEletronico bit NOT NULL,
	FazDistribuicao bit NOT NULL,
	IdOperadorResponsavelPonto int NULL,
	IdAvaliadorDiretor int NULL,
	SetorBatePonto bit NOT NULL,
	SetorAtivo bit NOT NULL,
	DataInclusao smalldatetime NOT NULL,
	IdSessao int NOT NULL,
	IdSessaoOperacao int NULL
	)  ON [PRIMARY]
GO
ALTER TABLE dbo.Tmp_Gen_Setor SET (LOCK_ESCALATION = TABLE)
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_SetorFazAnalise DEFAULT ((0)) FOR SetorFazAnalise
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_SetorTramita DEFAULT ((0)) FOR SetorTramita
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_SetorTramitaSetorExterno DEFAULT ((0)) FOR SetorTramitaSetorExterno
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_TrabalhaProcessoEletronico DEFAULT ((0)) FOR TrabalhaProcessoEletronico
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_FazDistribuicao DEFAULT ((0)) FOR FazDistribuicao
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_SetorBloqueado DEFAULT ((1)) FOR SetorAtivo
GO
ALTER TABLE dbo.Tmp_Gen_Setor ADD CONSTRAINT
	DF_Gen_Setor_DataInclusao DEFAULT (getdate()) FOR DataInclusao
GO
SET IDENTITY_INSERT dbo.Tmp_Gen_Setor ON
GO
IF EXISTS(SELECT * FROM dbo.Gen_Setor)
	 EXEC('INSERT INTO dbo.Tmp_Gen_Setor (IdSetor, IdDiretoria, CodigoSetor, NomeSetor, AreaSetor, SetorFazAnalise, SetorTramita, SetorTramitaSetorExterno, TrabalhaProcessoEletronico, FazDistribuicao, IdOperadorResponsavelPonto, IdAvaliadorDiretor, SetorBatePonto, SetorAtivo, DataInclusao, IdSessao, IdSessaoOperacao)
		SELECT IdSetor, IdDiretoria, CodigoSetor, NomeSetor, AreaSetor, SetorFazAnalise, SetorTramita, SetorTramitaSetorExterno, TrabalhaProcessoEletronico, FazDistribuicao, IdOperadorResponsavelPonto, IdAvaliadorDiretor, SetorBatePonto, SetorAtivo, DataInclusao, IdSessao, IdSessaoOperacao FROM dbo.Gen_Setor WITH (HOLDLOCK TABLOCKX)')
GO
SET IDENTITY_INSERT dbo.Tmp_Gen_Setor OFF
GO
ALTER TABLE dbo.Gen_UnidadeJurisdicionada
	DROP CONSTRAINT FK_Gen_UnidadeJurisdicionada_Gen_Setor_IdSetor
GO
ALTER TABLE dbo.Seg_OperadorInternoSetor
	DROP CONSTRAINT FK_Seg_OperadorInternoSetor_Gen_Setor_IdSetor
GO
ALTER TABLE dbo.Seg_AutorizacaoSetorPerfil
	DROP CONSTRAINT FK_Seg_AutorizacaoSetorPerfil_Gen_Setor_IdSetor
GO
ALTER TABLE dbo.Seg_OperadorInterno
	DROP CONSTRAINT FK_Seg_OperadorInterno_Gen_Setor_IdSetor_IdSetorLotacao
GO
DROP TABLE dbo.Gen_Setor
GO
EXECUTE sp_rename N'dbo.Tmp_Gen_Setor', N'Gen_Setor', 'OBJECT' 
GO
ALTER TABLE dbo.Gen_Setor ADD CONSTRAINT
	PK_Gen_Setor PRIMARY KEY CLUSTERED 
	(
	IdSetor
	) WITH( STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]

GO
CREATE UNIQUE NONCLUSTERED INDEX AK_Gen_Setor ON dbo.Gen_Setor
	(
	CodigoSetor
	) WITH( STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
GO
CREATE NONCLUSTERED INDEX IX_Gen_Setor_NomeSetor ON dbo.Gen_Setor
	(
	NomeSetor
	) WITH( STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
GO
ALTER TABLE dbo.Gen_Setor ADD CONSTRAINT
	FK_Gen_Setor_Gen_Diretoria_IdDiretoria FOREIGN KEY
	(
	IdDiretoria
	) REFERENCES dbo.Gen_Diretoria
	(
	IdDiretoria
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
ALTER TABLE dbo.Gen_Setor ADD CONSTRAINT
	FK_Gen_Setor_Gen_Setor_IdSessao FOREIGN KEY
	(
	IdSessao
	) REFERENCES dbo.Seg_Sessao
	(
	IdSessao
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Seg_OperadorInterno ADD CONSTRAINT
	FK_Seg_OperadorInterno_Gen_Setor_IdSetor_IdSetorLotacao FOREIGN KEY
	(
	IdSetorLotacao
	) REFERENCES dbo.Gen_Setor
	(
	IdSetor
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
ALTER TABLE dbo.Seg_OperadorInterno SET (LOCK_ESCALATION = TABLE)
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Seg_AutorizacaoSetorPerfil ADD CONSTRAINT
	FK_Seg_AutorizacaoSetorPerfil_Gen_Setor_IdSetor FOREIGN KEY
	(
	IdSetor
	) REFERENCES dbo.Gen_Setor
	(
	IdSetor
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
ALTER TABLE dbo.Seg_AutorizacaoSetorPerfil SET (LOCK_ESCALATION = TABLE)
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Seg_OperadorInternoSetor ADD CONSTRAINT
	FK_Seg_OperadorInternoSetor_Gen_Setor_IdSetor FOREIGN KEY
	(
	IdSetor
	) REFERENCES dbo.Gen_Setor
	(
	IdSetor
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
ALTER TABLE dbo.Seg_OperadorInternoSetor SET (LOCK_ESCALATION = TABLE)
GO
COMMIT
BEGIN TRANSACTION
GO
ALTER TABLE dbo.Gen_UnidadeJurisdicionada ADD CONSTRAINT
	FK_Gen_UnidadeJurisdicionada_Gen_Setor_IdSetor FOREIGN KEY
	(
	IdSetorResponsavel
	) REFERENCES dbo.Gen_Setor
	(
	IdSetor
	) ON UPDATE  NO ACTION 
	 ON DELETE  NO ACTION 
	
GO
ALTER TABLE dbo.Gen_UnidadeJurisdicionada SET (LOCK_ESCALATION = TABLE)
GO
COMMIT



In [ ]:
BEGIN TRAN

SET IDENTITY_INSERT Bdc.dbo.Gen_Setor ON
GO

  INSERT INTO Bdc.dbo.Gen_Setor
(
	IdSetor,
	IdDiretoria,
	CodigoSetor, 
	NomeSetor,
	SetorFazAnalise,
	SetorTramita,
	SetorTramitaSetorExterno,
	TrabalhaProcessoEletronico,
	FazDistribuicao,
	SetorBatePonto,
	SetorAtivo,
	DataInclusao,
	IdSessao
)
SELECT 
	 s.[idSetor]
	,IIF(s.IdDiretoria IS NULL, 0, s.IdDiretoria)
	,[codigo]
      ,[nome]
	  ,1
	  ,1
	  ,1
	  ,1
	  ,1
	  ,1
	  ,1
      ,GETDATE()
		,0
  FROM [Setor] s
LEFT JOIN Gen_Setor GS ON s.codigo = gs.CodigoSetor
--LEFT JOIN Gen_Setor GS ON s.IdSetor = gs.idSetor
WHERE ( CodigoSetor IS NULL )
ORDER BY Codigo


SET IDENTITY_INSERT Bdc.dbo.Gen_Setor OFF
GO

-- commit
-- rollback

In [ ]:
-- check


select	d.IdDiretoria, d.Sigla, d.NomeDiretoria,
		GD.IdDiretoria, gd.Sigla, gd.NomeDiretoria, 
		PF.NomePessoaFisica,
		PF2.NomePessoaFisica
from GenDiretoria D
LEFT JOIN Gen_Diretoria GD ON d.IdDiretoria = gd.IdDiretoria
LEFT JOIN Gen_PessoaFisica PF ON IdOperadorDiretor = IdPessoaFisica
LEFT JOIN Gen_PessoaFisica PF2 ON IdOperadorDiretorSubstituto = PF2.IdPessoaFisica

WHERe ( d.Sigla <> gd.Sigla ) OR ( d.IdDiretoria <> GD.IdDiretoria ) OR ( d.NomeDiretoria <> GD.NomeDiretoria )
ORDER BY d.Sigla

SELECT count(*) FROM GenDiretoria --d WHERE NOT EXISTS( SELECT * FROM Gen_Diretoria gd  WHERE d.IdDiretoria = gd.IdDiretoria )
SELECT * FROM Gen_Diretoria gd WHERE NOT EXISTS( SELECT * FROM GenDiretoria d  WHERE d.IdDiretoria = gd.IdDiretoria )


select	s.IdSetor, s.Codigo CodigoAR, s.nome NomeAR, s.DataInclusao, s.IdDiretoria IdDiretoriaAR,
		gs.IdSetor, CodigoSetor CodigoSetorBDC, NomeSetor, GS.IdDiretoria
from Setor S
LEFT JOIN Gen_Setor GS ON Codigo = CodigoSetor
WHERe 
--( s.nome <> NomeSetor ) 
--( CodigoSetor IS NULL )
--( s.IdDiretoria <> GS.IdDiretoria)
--( s.IdSetor <> GS.IdSetor)
ORDER BY Codigo